In [ ]:
GOOGLE_COLAB = True
path = ""
if GOOGLE_COLAB:
    from google.colab import drive, files
    drive.mount('/content/drive/')
    path = "/content/drive/My Drive/Colab Notebooks/car_dataset/"

Mounted at /content/drive/


In [ ]:
pwd

'/content'

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Flatten, Dense, MaxPool2D, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing.image import ImageDataGenerator , load_img
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.models import Model
import itertools
import os
import shutil
import random
import numpy as np
import glob
import matplotlib.pyplot as plt
import warnings
import pandas as pd

In [ ]:
dataset = path + "names.csv"
list_classes = pd.read_csv(dataset)
list_classes

In [ ]:
class_list = list_classes.values.tolist()  
class_list
new_list = []
for i in range(int(len(class_list))):
    r = class_list[i]
    new_list.append(r[0])


In [ ]:
test_path = path + "/train"
valid_path = path + "/valid"
train_path = path + "/test"

In [ ]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(224,224), classes= new_list, batch_size=16)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(224,224),classes= new_list, batch_size=1)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(224,224),classes= new_list, batch_size=8, shuffle=False)

Found 8000 images belonging to 196 classes.
Found 1546 images belonging to 196 classes.
Found 6489 images belonging to 196 classes.


In [ ]:
imgs, labels = next(train_batches)
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 10, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()
plotImages(imgs)
print(labels)

In [ ]:
resnet50_model = tf.keras.applications.resnet50.ResNet50()

102973440/102967424 [==============================] - 1s 0us/step


In [ ]:
resnet50_model.summary()

In [ ]:
base_model = ResNet50(include_top=False, weights='imagenet')
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(train_batches.num_classes, activation='softmax')(x)
model = Model(inputs=base_model.input , outputs=predictions)


94773248/94765736 [==============================] - 1s 0us/step


In [ ]:
fine_tune_at = 110
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
len(base_model.layers)

175

In [ ]:
model.summary()

In [ ]:
history = model.fit(train_batches, epochs=6)


Epoch 1/6
500/500 [==============================] - 948s 2s/step - loss: 1.7870 - accuracy: 0.6075
Epoch 2/6
500/500 [==============================] - 90s 179ms/step - loss: 0.4805 - accuracy: 0.9078
Epoch 3/6
500/500 [==============================] - 90s 179ms/step - loss: 0.1098 - accuracy: 0.9890
Epoch 4/6
500/500 [==============================] - 89s 178ms/step - loss: 0.0376 - accuracy: 0.9971
Epoch 5/6
500/500 [==============================] - 89s 179ms/step - loss: 0.0235 - accuracy: 0.9979
Epoch 6/6
500/500 [==============================] - 90s 180ms/step - loss: 0.0167 - accuracy: 0.9981


In [22]:
 history1 = model.evaluate(test_batches)
 

812/812 [==============================] - 1715s 2s/step - loss: 0.9986 - accuracy: 0.7369


In [38]:
model.save('final_model.h5',path)